In [3]:
import pandas as pd
import geopandas as gpd
import leafmap.foliumap as leafmapfol
import leafmap
import ast
import geopandas as gpd
from shapely.geometry import Polygon
import datacube
from odc.ui import with_ui_cbk
from deafrica_tools.plotting import rgb, display_map

%matplotlib inline
import matplotlib.pyplot as plt
from datacube.utils import geometry

from deafrica_tools.datahandling import load_ard
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.plotting import rgb, map_shapefile
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.classification import HiddenPrints
from odc.io.cgroups import get_cpu_quota
from deafrica_tools.classification import collect_training_data

import numpy as np
import xarray as xrc
from odc.algo import xr_reproject
from pyproj import Proj, transform
from datacube.utils.geometry import assign_crs
from datacube.testutils.io import rio_slurp_xarray
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.dask import create_local_dask_cluster
pd.set_option('display.max_colwidth', 500)
from odc.algo import xr_geomedian
import sys
sys.path.append('./utils/')
from data_cleaning import *
from feature_layers import *
import json

In [ ]:
!pip install leafmap

In [ ]:
!pip install openpyxl

## datacube init

In [4]:
dc = datacube.Datacube(app='training_data_extraction')

In [5]:
product = "s2_l2a"
measurements = dc.list_measurements()
measurements.loc[product]

,name,dtype,units,nodata,aliases,flags_definition
measurement,,,,,,
B01,B01,uint16,1,0.0,"[band_01, coastal_aerosol]",NaN
B02,B02,uint16,1,0.0,"[band_02, blue]",NaN
B03,B03,uint16,1,0.0,"[band_03, green]",NaN
B04,B04,uint16,1,0.0,"[band_04, red]",NaN
B05,B05,uint16,1,0.0,"[band_05, red_edge_1]",NaN
B06,B06,uint16,1,0.0,"[band_06, red_edge_2]",NaN
B07,B07,uint16,1,0.0,"[band_07, red_edge_3]",NaN
B08,B08,uint16,1,0.0,"[band_08, nir, nir_1]",NaN
B8A,B8A,uint16,1,0.0,"[band_8a, nir_narrow, nir_2]",NaN


## Read vector file

In [6]:
crop_vectors = gpd.read_file('../Numeric data models/data/crop_vectors_v4.geojson')

In [5]:
crop_vectors.culture.value_counts()

olive             779
blé tendre        385
blé dur           332
fèves             181
orge              167
pomme de terre    143
pomme             136
maraichage        126
pois chiche       106
Name: culture, dtype: int64

In [5]:
raw_crop_plant_harvest_df = pd.read_excel('./data/Crop_Calendar_Data.xlsx')
crop_plant_harvest_df = get_crop_planting_harvest_info(raw_crop_plant_harvest_df)

In [7]:
crop_plant_harvest_df

,Crop,early_sowing_day,early_sowing_month,later_sowing_day,later_sowing_month,early_harvest_day,early_harvest_month,late_harvest_day,late_harvest_month
0,"Wheat, durum",15,11,15,12,15,6,10,7
1,"Wheat, durum",20,10,15,11,1,6,20,6
2,"Wheat, durum",20,10,15,12,1,6,20,7
3,"Wheat, durum",1,11,20,11,1,6,15,6
4,"Wheat, durum",15,11,30,12,30,6,10,7
5,"Wheat, durum",15,11,30,12,30,6,10,7
6,"Wheat, bread",15,11,15,12,15,6,10,7
7,"Wheat, bread",20,10,15,11,1,6,20,6
8,"Wheat, bread",20,10,15,12,1,6,10,7
9,"Wheat, bread",1,11,20,11,1,6,15,6


| Crop | (planting month, harvesting month)  | Details  |
|-----|---|---|
|Blé tendre  |  (10,6) |   |
|Blé dur     | (10,6)  |   |
|Orge     |  (11,5) |   |
|Fèves  |  (11,5) |   |
|Pois chiche     | (11,(4,5))  |   |
|Pomme de terre     | (wide range all year crop)  |   |
|Maraichage     | (wide range all year crop)|   |

## feature layers queries

* S (stands for semester geomedian)
* Q (stands for quarter geomedian)
* w (stands for weekly groupby series)

| Experiment name | type of model  | features_preview  |Number of models|Experiment id
|-----|---|---|---|---|
|year (example:2019) geomedian  |  Annual mapping | reflectance{i}_S1, reflectance{i}_S2  |1|gm_1|
|October 18 July 19 custom geomedian     | Annual mapping  |reflectance{i}_S1, reflectance{i}_S2   |1|cus_gm_1|
|October 18 July 19 custom geomedian    | Annual mapping  |reflectance{i}_Q1, reflectance{i}_Q2,reflectance{i}_Q3, reflectance{i}_Q4  |1|cus1_gm_1|
|October18 December18 / Previous + January19 March19 / Previous* + April July  custom geomedians| In season mapping  |   |3|cus_gm_3|
|indices TS   | Annual or in season |reflectance{i}_w1, reflectance{i}_w2,reflectance{i}_w3, reflectance{i}_w4 ...... ||ts_grpw_|

##### Annual mapping

In [6]:
#QUERY
time_range = ('2019')
measurements = ["blue", "green", "red", "nir", "swir_1", "swir_2", "red_edge_1", "red_edge_2", "red_edge_3", "sdev", "bcdev", "edev"]
resolution = (-10,10)
output_crs = 'EPSG:6933'
query_gm_1 = {'time': time_range, 
              'measurements': measurements, 
              'resolution': resolution, 
              'output_crs': output_crs, 
              'group_by' : 'solar_day', 
              'resampling': 'bilinear'}

In [9]:
#QUERY
time_range = ('2018', '2019')
measurements = ["blue", "green", "red", "nir", "swir_1", "swir_2", "red_edge_1", "red_edge_2", "red_edge_3", "sdev", "bcdev", "edev"]
resolution = (-10,10)
output_crs = 'EPSG:6933'
query_gm1_1 = {'time': time_range, 
               'measurements': measurements, 
               'resolution': resolution, 
               'output_crs': output_crs, 
               'group_by' : 'solar_day', 
               'resampling': 'bilinear'}

In [10]:
#QUERY
time_range = ('2018-10-01', '2019-07-31')
measurements = ["blue", "green", "red", "nir", "swir_1", "swir_2", "red_edge_1", "red_edge_2", "red_edge_3"]
resolution = (-10,10)
output_crs = 'EPSG:6933'
query_cus1_gm_1 = {'time': time_range, 
                   'measurements': measurements, 
                   'resolution': resolution, 
                   'output_crs': output_crs, 
                   'group_by' : 'solar_day', 
                   'resampling': 'bilinear'}

##### In season mapping

In [11]:
#QUERY
time_range = ('2018-10-01', '2018-12-31')
measurements = ["blue", "green", "red", "nir", "swir_1", "swir_2", "red_edge_1", "red_edge_2", "red_edge_3"]
resolution = (-10,10)
output_crs = 'EPSG:6933'
query_cus_gm_3 = {'time': time_range, 
                   'measurements': measurements, 
                   'resolution': resolution, 
                   'output_crs': output_crs, 
                   'group_by' : 'solar_day', 
                   'resampling': 'bilinear'}

In [12]:
#QUERY
time_range = ('2018-10-01', '2019-03-31')
measurements = ["blue", "green", "red", "nir", "swir_1", "swir_2", "red_edge_1", "red_edge_2", "red_edge_3"]
resolution = (-10,10)
output_crs = 'EPSG:6933'
query_cus1_gm_3 = {'time': time_range, 
                   'measurements': measurements, 
                   'resolution': resolution, 
                   'output_crs': output_crs, 
                   'group_by' : 'solar_day', 
                   'resampling': 'bilinear'}

In [13]:
#QUERY
time_range = ('2018-10-01', '2019-07-31')
measurements = ["blue", "green", "red", "nir", "swir_1", "swir_2", "red_edge_1", "red_edge_2", "red_edge_3"]
resolution = (-10,10)
output_crs = 'EPSG:6933'
query_cus2_gm_3 = {'time': time_range, 
                   'measurements': measurements, 
                   'resolution': resolution, 
                   'output_crs': output_crs, 
                   'group_by' : 'solar_day', 
                   'resampling': 'bilinear'}

##### TS mapping

In [14]:
#QUERY
time_range = ('2018-09-01', '2019-07-31')
measurements = ["blue", "green", "red", "nir", "swir_1", "swir_2", "red_edge_1", "red_edge_2", "red_edge_3", "mask"]
resolution = (-10,10)
output_crs = 'EPSG:6933'
query_ts_grpw_ = {'time': time_range, 
                   'measurements': measurements, 
                   'resolution': resolution, 
                   'output_crs': output_crs, 
                   'group_by' : 'solar_day'}

## Save queries

In [123]:
import json
with open('models_training_data_queries.json', 'w') as fp:
    json.dump({'query_gm_1': query_gm_1, 
               'query_gm1_1': query_gm1_1,
              'query_cus1_gm_1':query_cus1_gm_1,
              'query_cus_gm_3':query_cus_gm_3,
              'query_cus1_gm_3':query_cus1_gm_3,
              'query_cus2_gm_3':query_cus2_gm_3,
              'query_ts_grpw_':query_ts_grpw_}, fp)

## Extracting multispectral data

In [7]:
queries = json.load(open('models_training_data_queries.json'))
crop_vectors_samples = crop_vectors.groupby('field').sample(frac=0.5).reset_index().drop('index', axis=1)

In [9]:
%%time
#zonal_stats = 'median'
return_coords = True
field = 'field'
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

column_names, array_values = collect_training_data(gdf=crop_vectors_samples,
                                                    dc_query=query_gm_1,
                                                    ncpus=ncpus,
                                                    clean=False,
                                                    return_coords=return_coords,
                                                    field=field,
                                                    #zonal_stats=zonal_stats,
                                                    fail_threshold=0.0075,
                                                    feature_func= gm_1)

ncpus = 4


  0%|          | 0/1178 [00:00<?, ?it/s]

Percentage of possible fails after run 1 = 0.0 %
Returning data without cleaning
Output shape:  (198359, 37)
CPU times: user 9.93 s, sys: 881 ms, total: 10.8 s
Wall time: 17min 39s


In [17]:
pd.DataFrame(array_values, columns=column_names).to_parquet('./data/gm_1.parquet')

In [ ]:
%%time
#zonal_stats = 'median'
return_coords = True
field = 'field'
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

column_names, array_values = collect_training_data(gdf=crop_vectors_samples,
                                                    dc_query=queries['query_cus1_gm_1'],
                                                    ncpus=15,
                                                    clean=False,
                                                    return_coords=return_coords,
                                                    field=field,
                                                    #zonal_stats=zonal_stats,
                                                    fail_threshold=0.0075,
                                                    feature_func= cus1_gm_1)

In [10]:
%%time
#zonal_stats = 'median'
return_coords = True
field = 'field'
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

column_names, array_values = collect_training_data(gdf=crop_vectors.head(),
                                                    dc_query=queries['query_cus_gm_3'],
                                                    ncpus=15,
                                                    clean=False,
                                                    return_coords=return_coords,
                                                    field=field,
                                                    #zonal_stats=zonal_stats,
                                                    fail_threshold=0.0075,
                                                    feature_func= cus_gm_3)

ncpus = 4


  0%|          | 0/5 [00:00<?, ?it/s]

Percentage of possible fails after run 1 = 0.0 %
Returning data without cleaning
Output shape:  (665, 29)
CPU times: user 99.1 ms, sys: 112 ms, total: 211 ms
Wall time: 36.7 s


In [11]:
column_names

['field',
 'blue_s2_Q0_n_1',
 'green_s2_Q0_n_1',
 'red_s2_Q0_n_1',
 'nir_s2_Q0_n_1',
 'swir_1_s2_Q0_n_1',
 'swir_2_s2_Q0_n_1',
 'red_edge_1_s2_Q0_n_1',
 'red_edge_2_s2_Q0_n_1',
 'red_edge_3_s2_Q0_n_1',
 'NDVI_s2_Q0_n_1',
 'EVI_s2_Q0_n_1',
 'SAVI_s2_Q0_n_1',
 'NDMI_s2_Q0_n_1',
 'blue_s2_Q1_n_1',
 'green_s2_Q1_n_1',
 'red_s2_Q1_n_1',
 'nir_s2_Q1_n_1',
 'swir_1_s2_Q1_n_1',
 'swir_2_s2_Q1_n_1',
 'red_edge_1_s2_Q1_n_1',
 'red_edge_2_s2_Q1_n_1',
 'red_edge_3_s2_Q1_n_1',
 'NDVI_s2_Q1_n_1',
 'EVI_s2_Q1_n_1',
 'SAVI_s2_Q1_n_1',
 'NDMI_s2_Q1_n_1',
 'x_coord',
 'y_coord']

In [ ]:
%%time
#zonal_stats = 'median'
return_coords = True
field = 'field'
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

column_names, array_values = collect_training_data(gdf=crop_vectors,
                                                    dc_query=queries['query_cus1_gm_3'],
                                                    ncpus=15,
                                                    clean=False,
                                                    return_coords=return_coords,
                                                    field=field,
                                                    #zonal_stats=zonal_stats,
                                                    fail_threshold=0.0075,
                                                    feature_func= cus1_gm_3)

In [ ]:
%%time
#zonal_stats = 'median'
return_coords = True
field = 'field'
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

column_names, array_values = collect_training_data(gdf=crop_vectors,
                                                    dc_query=queries['query_cus2_gm_3'],
                                                    ncpus=15,
                                                    clean=False,
                                                    return_coords=return_coords,
                                                    field=field,
                                                    #zonal_stats=zonal_stats,
                                                    fail_threshold=0.0075,
                                                    feature_func= cus2_gm_3)